In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
from vllm import LLM

In [ ]:
from src.zeroband.inference.pipeline import PipelineConfig, patch_model_load, setup_comm, setup_hooks

config = PipelineConfig(
    rank=0,
    world_size=2,
    iroh_seed=0,
    iroh_peer_id="ff87a0b0a3c7c0ce827e9cada5ff79e75a44a0633bfcb5b50f99307ddb26b337",
    connection_num_retries=3,
)

node = setup_comm(config)
patch_model_load(config)

In [ ]:
llm = LLM(
    model="Qwen/Qwen3-0.6B",
    tensor_parallel_size=1,
    max_seq_len_to_capture=16384,
    max_model_len=16384,
    quantization=None,
    enforce_eager=True,
    disable_async_output_proc=True,
    download_dir="/alloc",
    dtype="bfloat16")

In [4]:
setup_hooks(config, llm, node)

In [ ]:
from vllm import SamplingParams

sampling_params = SamplingParams(
    max_tokens=1,
    temperature=0.7,
    top_p=0.9,
    top_k=40,
    seed=42,
)
request_outputs = llm.generate("Hello, world!", sampling_params)
print(request_outputs[0].outputs[0].text)